# Install

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install openai

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load

In [6]:
import sys
sys.path.append('/content/drive/MyDrive/MemeProject/src/pytorch/augmentation')
print(sys.path)

['/content', '/env/python', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.9/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/MemeProject/src/pytorch/augmentation']


In [7]:
import re
import config as cfg
import pandas as pd
from pshmodule.utils import filemanager as fm

In [8]:
df = fm.load(cfg.userdict_aug)

extension : .pickle
Loaded 121407 records from drive/MyDrive/MemeProject/data/augmentation/userdict_aug.pickle


# Reshape

In [9]:
df = df[['u', 'label']]

In [10]:
df.head()

,u,label
0,나 다시는 라면 안 먹을 거야!,98
1,와타시 다시는 라면 안 먹을 거야!,98
2,와따시 다시는 라면 안 먹을 거야!,98
3,나 중간고사 반에서 1등했어,147
4,와타시 중간고사 반에서 1등했어,147


In [11]:
df.shape

(121407, 2)

In [12]:
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

In [13]:
df.shape

(108584, 2)

In [14]:
count = {i[1]['label']: 0 for i in df.iterrows()}

for i in df.iterrows():
  count[i[1]['label']] += 1

In [18]:
temp = dict(sorted(count.items(), key=lambda x: x[1], reverse=True))

In [19]:
temp

{18: 2319,
 147: 1764,
 57: 1699,
 110: 1598,
 51: 1581,
 58: 1525,
 108: 1519,
 145: 1517,
 106: 1441,
 16: 1424,
 28: 1334,
 49: 1311,
 66: 1170,
 111: 1129,
 73: 1114,
 13: 1052,
 44: 1025,
 94: 1019,
 128: 1008,
 139: 997,
 65: 976,
 74: 970,
 53: 970,
 146: 964,
 64: 962,
 63: 954,
 54: 945,
 77: 934,
 93: 926,
 102: 923,
 96: 916,
 144: 908,
 27: 902,
 41: 891,
 140: 888,
 31: 879,
 107: 873,
 100: 869,
 33: 862,
 71: 858,
 119: 855,
 36: 855,
 42: 848,
 134: 842,
 141: 828,
 46: 825,
 0: 819,
 78: 812,
 142: 796,
 148: 790,
 121: 789,
 45: 772,
 136: 771,
 6: 770,
 80: 768,
 92: 765,
 9: 763,
 81: 763,
 34: 756,
 38: 749,
 95: 736,
 101: 734,
 117: 728,
 143: 725,
 47: 720,
 85: 719,
 72: 719,
 151: 718,
 152: 712,
 62: 705,
 98: 704,
 153: 704,
 150: 700,
 8: 691,
 2: 691,
 50: 678,
 68: 672,
 67: 671,
 17: 663,
 32: 659,
 48: 651,
 114: 649,
 115: 644,
 89: 642,
 60: 630,
 39: 629,
 4: 629,
 20: 626,
 75: 620,
 120: 619,
 109: 616,
 99: 616,
 52: 614,
 125: 607,
 19: 599,
 82:

# Under sampling

1000건 기준 나누기

In [36]:
limit = 1000

In [37]:
over_dict = {i[0]: i[1] for i in temp.items() if i[1] >= limit}
under_dict = {i[0]: i[1] for i in temp.items() if i[1] < limit}

In [38]:
over_dict_temp = [[i[1]['u'], i[1]['label']] for i in df.iterrows() if i[1]['label'] in over_dict.keys()]
under_dict_temp = [[i[1]['u'], i[1]['label']] for i in df.iterrows() if i[1]['label'] in under_dict.keys()]

In [39]:
df_over = pd.DataFrame(over_dict_temp, columns=['u', 'label'])
df_under = pd.DataFrame(under_dict_temp, columns=['u', 'label'])

In [40]:
df_over.shape

(26549, 2)

In [41]:
df_under.shape

(82035, 2)

1000건 이상 자르기

In [46]:
result = []
for i in over_dict.items():
  globals()['df_{}'.format(i[0])] = df_over[df_over.label==i[0]]
  globals()['df_{}'.format(i[0])].sample(frac=1).reset_index(drop=True, inplace=True)
  for i in globals()['df_{}'.format(i[0])].iloc[:limit].iterrows():
    result.append([i[1]['u'], i[1]['label']])

In [48]:
df_result = pd.DataFrame(result, columns=['u', 'label'])

In [49]:
df_result.shape

(19000, 2)

In [51]:
count = {i[1]['label']: 0 for i in df_result.iterrows()}

for i in df_result.iterrows():
  count[i[1]['label']] += 1

In [52]:
temp = dict(sorted(count.items(), key=lambda x: x[1], reverse=True))

In [54]:
temp

{18: 1000,
 147: 1000,
 57: 1000,
 110: 1000,
 51: 1000,
 58: 1000,
 108: 1000,
 145: 1000,
 106: 1000,
 16: 1000,
 28: 1000,
 49: 1000,
 66: 1000,
 111: 1000,
 73: 1000,
 13: 1000,
 44: 1000,
 94: 1000,
 128: 1000}

In [55]:
df_temp = pd.concat([df_result, df_under])

In [56]:
df_temp.reset_index(drop=True, inplace=True)

In [57]:
df_temp.head()

,u,label
0,나 크리스마스에 고백할 거야,18
1,나 12월 25일에 고백할 거야,18
2,나 크리스마스 때 고백할 거야,18
3,나 성탄절날 고백할 거야,18
4,나 클스마스에 고백할 거야,18


In [58]:
count = {i[1]['label']: 0 for i in df_temp.iterrows()}

for i in df_temp.iterrows():
  count[i[1]['label']] += 1

In [59]:
count

{18: 1000,
 147: 1000,
 57: 1000,
 110: 1000,
 51: 1000,
 58: 1000,
 108: 1000,
 145: 1000,
 106: 1000,
 16: 1000,
 28: 1000,
 49: 1000,
 66: 1000,
 111: 1000,
 73: 1000,
 13: 1000,
 44: 1000,
 94: 1000,
 128: 1000,
 98: 704,
 39: 629,
 129: 357,
 87: 342,
 32: 659,
 19: 599,
 107: 873,
 125: 607,
 70: 432,
 25: 582,
 74: 970,
 117: 728,
 14: 333,
 11: 522,
 88: 434,
 118: 546,
 120: 619,
 91: 561,
 43: 542,
 6: 770,
 22: 459,
 53: 970,
 82: 596,
 115: 644,
 54: 945,
 84: 247,
 27: 902,
 15: 571,
 140: 888,
 90: 228,
 26: 558,
 137: 376,
 124: 484,
 24: 330,
 5: 200,
 1: 535,
 127: 517,
 109: 616,
 122: 317,
 55: 251,
 103: 467,
 130: 263,
 61: 589,
 10: 460,
 116: 374,
 85: 719,
 40: 471,
 64: 962,
 37: 519,
 136: 771,
 149: 368,
 72: 719,
 139: 997,
 45: 772,
 76: 354,
 65: 976,
 95: 736,
 105: 298,
 131: 202,
 133: 341,
 29: 334,
 141: 828,
 114: 649,
 77: 934,
 23: 214,
 38: 749,
 112: 309,
 99: 616,
 79: 168,
 93: 926,
 56: 137,
 30: 6,
 71: 858,
 69: 506,
 86: 133,
 113: 48,
 35:

# Save

In [60]:
df_temp.rename(columns={'u':'content'}, inplace=True)

In [61]:
fm.save(cfg.train_aug_data, df_temp)

Saved 101035 records
